<div style="position: relative; text-align: center; padding: 30px;">
  <h1><strong>Gestión en Logística y Cadena de Suministro</strong></h1>
  <h3><strong>Ejercicio 3</strong></h3>
</div>

Un fabricante de equipos de aire acondicionado ha experimentado un incremento significativo en la demanda de dichos equipos en algunas zonas de los Estados Unidos. La compañía anticipa una demanda total para el próximo año de $d_j$ unidades para la zona geográfica $j$ del país, que está dividido en $n$ zonas de demanda.  

La gerencia está considerando el diseño de una red de manufactura y ha seleccionado $m$ sitios potenciales para ubicar plantas productivas. Se puede abrir una sola planta en cada sitio. Dichas plantas pueden tener una capacidad de producción de $Q_1$ o de $Q_2$ unidades.  

Para cada posible sitio potencial se especifican dos costos fijos anuales de operación $f_{1i}$ y $f_{2i}$ para $i = 1, \dots, m$.  
- $f_{1i}$ corresponde a la selección del sitio para ubicar una planta con capacidad de $Q_1$ unidades anuales.  
- $f_{2i}$ corresponde a la selección del sitio para ubicar una planta con una capacidad de $Q_2$ unidades anuales.  

Sea también $c_{ij}$, para $i = 1, \dots, m$ y $j = 1, \dots, n$, el costo de producción y transporte por unidad del sitio $i$ a la zona $j$.  

Se requiere formular el problema para determinar lo siguiente: 

- ¿En cuáles sitios se debe abrir una planta productiva y con qué capacidad?  
- ¿Cuántos equipos deben enviarse desde cada una de las plantas productivas a cada una de las zonas de demanda? 
- ¿A cuánto asciende el costo total?

In [1]:
from ortools.linear_solver import pywraplp

In [2]:
solver = pywraplp.Solver.CreateSolver('SCIP')

In [3]:
epsilon = 1e-6

### **Conjunto de índices**

- $M$: Conjunto de sitios potenciales para ubicar plantas.
- $N$: Conjunto de zonas de demanda.



In [4]:
n = 4  #no. de zonas
m = 4  #posible construcción
N = range(n)
M = range(m)

### **Parámetros**

- $d_j$ o $d_j$ para $j=1,\ldots,n$: Demanda anual de la zona $j$.
- $Q_1$ y $Q_2$: Capacidades anuales posibles de una planta.
- $f_{1i}$: Costo fijo anual si se abre la planta en el sitio $i$ con capacidad $Q_1$.
- $f_{2i}$: Costo fijo anual si se abre la planta en el sitio $i$ con capacidad $Q_2$.
- $c_{ij}$: Costo de producción y transporte por unidad desde el sitio $i$ a la zona $j$.




In [5]:
# demanda
d = [180000, 120000, 110000, 100000]

# cantidades
Q1 = 200000
Q2 = 400000

# costos fijos
f1 = [6300000, 5500000, 5600000, 6100000]
f2 = [10000000, 8200000, 9300000, 10200000]

# costos variables
c = [
    [211, 232, 240, 300],
    [232, 212, 230, 280],
    [238, 230, 215, 270],
    [299, 280, 270, 225]
]

### **Variables de decisión**

Variables de asignación (flujos):  
$$
x_{ij} \ge 0: \text{Número de equipos enviados desde la planta ubicada en el sitio $i$ a la zona $j$}
$$

In [6]:
x = {}
for i in M: #potenciales
    for j in N: #zonas
        x[i, j] = solver.NumVar(0, solver.infinity(), f'x_{i}_{j}')

Variables binarias de selección de planta:  
Utilizamos dos variables binarias para cada sitio $i$:
- $y_{1i} \in \{0,1\}$: Toma el valor 1 si se abre la planta en el sitio $i$ con capacidad $Q_1$; 0 en caso contrario.
- $y_{2i} \in \{0,1\}$: Toma el valor 1 si se abre la planta en el sitio $i$ con capacidad $Q_2$; 0 en caso contrario.

In [7]:
y1 = {}
y2 = {}
for i in M: #potenciales
    y1[i] = solver.BoolVar(f'y1_{i}')
    y2[i] = solver.BoolVar(f'y2_{i}')

### **Función objetivo**

El costo total se compone de:
- **Costos fijos:** Si se abre la planta en el sitio $i$ con capacidad $Q_1$ se incurre en un costo de $f_{1i}$; si se abre con capacidad $Q_2$, se incurre en $f_{2i}$.
- **Costos variables:** Por cada unidad enviada desde el sitio $i$ a la zona $j$ se paga $c_{ij}$.


Así, la función objetivo (minimizar el costo total) es:

$$
\min \quad \sum_{i=1}^{m} \Big( f_{1i}\,y_{1i} + f_{2i}\,y_{2i} \Big) + \sum_{i=1}^{m} \sum_{j=1}^{n} c_{ij}\,x_{ij}.
$$


In [8]:
costo = solver.Objective()

costo = (
    sum(f1[i] * y1[i] + f2[i] * y2[i] for i in M) # costos fijos
    + sum(c[i][j] * x[i, j] for i in M for j in N) #  costos variables
)

solver.Minimize(costo)

### **Restricciones**

**Satisfacción de la demanda**

Cada zona $j$ debe recibir al menos la cantidad demandada $d_j$:

$$
\sum_{i=1}^{m} x_{ij} \ge d_j, \quad \forall \, j = 1, \ldots, n.
$$


In [9]:
for j in N: #zonas
    solver.Add(sum(x[i, j] for i in M) >= d[j])

**Restricción de capacidad en cada planta**

La cantidad producida (y enviada) desde cada planta $i$ no puede exceder la capacidad de la planta, la cual depende de la decisión tomada en dicho sitio. Es decir, para cada sitio $i$:

$$
\sum_{j=1}^{n} x_{ij} \le Q_1\,y_{1i} + Q_2\,y_{2i}, \quad \forall \, i = 1, \ldots, m.
$$


In [10]:
for i in M: #posibles
    solver.Add(sum(x[i, j] for j in N) <= Q1 * y1[i] + Q2 * y2[i])

**A lo sumo una planta por sitio**

En cada sitio se puede seleccionar como máximo una de las dos opciones (o ninguna):

$$
y_{1i} + y_{2i} \le 1, \quad \forall \, i = 1, \ldots, m.
$$

In [11]:
for i in M: #posibles
    solver.Add(y1[i] + y2[i] <= 1)

**No negatividad y dominio de variables**

$$
x_{ij} \ge 0, \quad \forall \, i = 1, \ldots, m,\; \forall \, j = 1, \ldots, n,
$$
$$
y_{1i}, \, y_{2i} \in \{0,1\}, \quad \forall \, i = 1, \ldots, m.
$$

### **Resolver**

In [12]:
solver.Solve()

0

In [13]:
print(f'Costo total = {solver.Objective().Value()}')

Costo total = 129700000.0


In [14]:
for i in M: #posibles a abrir
    if y1[i].solution_value() == 1: # si es 1, se abre
        print(f'Ciudad {i+1}: Abrir planta con capacidad {Q1} (costo fijo = {f1[i]})')
    elif y2[i].solution_value() == 1: # si es 1, se abre
        print(f'Ciudad {i+1}: Abrir planta con capacidad {Q2} (costo fijo = {f2[i]})')
    else:
        print(f'Ciudad {i+1}: No se abre planta.')

Ciudad 1: No se abre planta.
Ciudad 2: Abrir planta con capacidad 400000 (costo fijo = 8200000)
Ciudad 3: No se abre planta.
Ciudad 4: Abrir planta con capacidad 200000 (costo fijo = 6100000)


In [15]:
for i in M: #distribucción :)
    for j in N:
        cantidad = x[i, j].solution_value()
        if cantidad > epsilon:  # Mostrar solo flujos significativos
            print(f'Envío desde Ciudad {i+1} a Zona {j+1}: {cantidad} unidades')

Envío desde Ciudad 2 a Zona 1: 180000.0 unidades
Envío desde Ciudad 2 a Zona 2: 119999.99999999999 unidades
Envío desde Ciudad 2 a Zona 3: 100000.0 unidades
Envío desde Ciudad 4 a Zona 3: 9999.999999999987 unidades
Envío desde Ciudad 4 a Zona 4: 100000.0 unidades
